In [1]:
import pyvisa
rm = pyvisa.ResourceManager()
rm.list_resources()

('TCPIP0::A-34461A-10522.local::inst0::INSTR',
 'TCPIP0::A-34461A-10532.local::inst0::INSTR',
 'TCPIP0::A-B2962A-50476.local::hislip0::INSTR',
 'TCPIP0::A-B2962A-50476.local::inst0::INSTR',
 'ASRL1::INSTR',
 'ASRL3::INSTR',
 'ASRL4::INSTR',
 'ASRL5::INSTR',
 'GPIB0::13::INSTR')

In [2]:
inst = rm.open_resource('GPIB0::13::INSTR')

In [145]:
inst.query('*IDN?')

'YOKOGAWA,GS210,91M313872,1.05\n'

In [188]:
inst.write(':output off') # on / 1, off / 0

13

In [169]:
inst.query(':output?')

'0\n'

In [5]:
inst.write(':source:function voltage')

26

In [6]:
inst.query(':source:function?')

'VOLT\n'

In [7]:
inst.write(':source:level:fix .01')

23

18

In [449]:
inst.write(':source:range 30')

18

In [35]:
float(inst.query(':source:range?'))

10.0

In [177]:
inst.read_termination = '\n'

In [442]:
print(inst.query('*IDN?'))
inst.write('*RST')
inst.write(':source:function voltage')
inst.write(':source:protection:current 0.001')
inst.write(':source:level:fix 0.1')

YOKOGAWA,GS210,91M313872,1.05


23

In [443]:
inst.write(':sense:state 1')
inst.write(':sense:nplc 1')
inst.write(':sense:delay 0')
inst.write(':sense:trigger IMM')
inst.write(':sense:zero:execute')
inst.write(':output on') # on / 1, off / 0

12

In [444]:
float(inst.query(':fetch?'))

0.00077

In [445]:

inst.write(':output off') # on / 1, off / 0

13

In [446]:
inst.write(':trace:data:format ASCII')

26

In [447]:
inst.query(':TRACe:DATA:READ? TM')

VisaIOError: VI_ERROR_TMO (-1073807339): Timeout expired before operation completed.

In [452]:
inst.write(':source:level:fix -30')

23

In [434]:
import numpy as np
possible_ranges = np.array([30, 10, 1, .1, .01])

In [440]:
voltage = 7
autorange = np.copy(possible_ranges)
autorange[autorange < voltage] = np.nan
print(autorange)
np.nanmin(autorange)

[30. 10. nan nan nan]


10.0

In [15]:
from p5control import InstrumentGateway

In [16]:
gw = InstrumentGateway()
gw.connect()
gw.motor.set_target_position(13)
gw.motor.set_target_speed(7000)
gw.motor.set_moving(True)

In [23]:
gw.motor.get_status()

{'time': 1716455059.4148033,
 'position': 0.0,
 'moving': False,
 'temperature': 30}

In [13]:
from p5control import DataGateway, InstrumentGateway
from time import sleep

In [27]:
gw = InstrumentGateway()
gw.connect()

dgw = DataGateway()
dgw.connect()


In [15]:
try:
    i = 0
    while True:
        gw.gate.setOutput(True)
        gw.gate.setVoltage((i%2)*1e-3)
        gw.gate.setOutput(False)
        i +=1
except KeyboardInterrupt:
    pass

In [16]:
try:
    i = 0
    m = gw.measure()
    m.start()
    while True:
        gw.gate.setOutput(True)
        gw.gate.setVoltage((i%2)*1e-3)
        gw.gate.setOutput(False)
        i +=1
except KeyboardInterrupt:
    m.stop()

In [75]:
try:
    i = 0
    m = gw.measure()
    while True:
        m.start()
        gw.gate.setOutput(True)
        gw.gate.setVoltage((i%2)*1e-3)
        i +=1
        m.stop()
        gw.gate.setOutput(False)
except KeyboardInterrupt:
    m.stop()

EOFError: connection closed by peer

In [72]:
from time import time
try:
    i=0
    while True:
        i +=1
        tic = time()
        gw.gate.write(f':output {i%2}')
        gw.gate.query('*OPC?')
        toc = time()
        print(toc-tic, i%2, end='\r')
except KeyboardInterrupt:
    pass

In [69]:
4%2

0